<a href="https://colab.research.google.com/github/crudolphMQU/programmingbitcoin/blob/master/code-ch02/Chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# set up and imports

In [1]:
!git clone https://github.com/crudolphMQU/programmingbitcoin

Cloning into 'programmingbitcoin'...
remote: Enumerating objects: 6450, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 6450 (delta 32), reused 15 (delta 15), pack-reused 6406 (from 2)
Receiving objects: 100% (6450/6450), 38.93 MiB | 19.25 MiB/s, done.
Resolving deltas: 100% (4282/4282), done.


In [2]:
ls

programmingbitcoin/  sample_data/


In [3]:
cd programmingbitcoin/code-ch02

/content/programmingbitcoin/code-ch02


In [4]:
!pip install virtualenv==20.17.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 23.3 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.3.7
    Uninstalling platformdirs-4.3.7:
      Successfully uninstalled platformdirs-4.3.7


In [5]:
!pip install -r /content/programmingbitcoin/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: jupyter-server
    Found existing installation: jupyter-server 1.16.0
    Uninstalling jupyter-server-1.16.0:
      Successfully uninstalled jupyter-server-1.16.0


In [6]:
!pip install helper

  Preparing metadata (setup.py) ... done
  Created wheel for helper: filename=helper-2.5.0-py2.py3-none-any.whl size=19169 sha256=2b416e0ab968b611c71e3b56c4af7bcb544e35db997a5fce9f5a11d9905f9553
  Stored in directory: /root/.cache/pip/wheels/8b/ab/a7/06bcd4f2d89113bb7b0caaf5562aa209a27863d8cccb07a1bc
Successfully built helper


In [7]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
import helper

from helper import run
import ecc

from ecc import Point

Using the curve in fig 2-9

$y^2 = x^3 + 5x + 7$

defined in general form in the `ecc.py` file, we're interested in the following points:

In [8]:
from ecc import Point
p1 = Point(-1, -1, 5, 7)
# p2 = Point(-1, -2, 5, 7)

running p2 will raise the ValueError from the ecc file advising the point is not on the curve. `

# Exercise 1

Determine which of these points are on the curve \\(y^{2}\\)=\\(x^{3}\\)+5x+7:

(2,4), (-1,-1), (18,77), (5,7)

In [9]:
# Exercise 1

# (2,4), (-1,-1), (18,77), (5,7)
# equation in python is: y**2 == x**3 + 5*x + 7

In [10]:
# Curve parameters
a = 5
b = 7

# List of points to check
points = [(2, 4), (-1, -1), (18, 77), (5, 7)]

# Check each point
for x, y in points:
    left = y**2
    right = x**3 + a*x + b
    if left == right:
        print(f"Point ({x}, {y}) is ON the curve.")
    else:
        print(f"Point ({x}, {y}) is NOT on the curve.")


Point (2, 4) is NOT on the curve.
Point (-1, -1) is ON the curve.
Point (18, 77) is ON the curve.
Point (5, 7) is NOT on the curve.


# Exercise 2

Write the `__ne__` method for `Point`.

#### Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_ne`

In [15]:
import importlib
import ecc
importlib.reload(ecc)

<module 'ecc' from '/content/programmingbitcoin/code-ch02/ecc.py'>

issue: running notebook with new kernel loads old, unedited, ecc file.

In [17]:
ls

answers.py      ecc.py       helper.py    __pycache__/
Chapter2.ipynb  examples.py  jupyter.txt


In [16]:
# Exercise 2

reload(ecc)
run(ecc.PointTest("test_ne"))

E
ERROR: test_ne (ecc.PointTest.test_ne)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/content/programmingbitcoin/code-ch02/ecc.py", line 188, in test_ne
    self.assertTrue(a != b)
                    ^^^^^^
  File "/content/programmingbitcoin/code-ch02/ecc.py", line 145, in __ne__
    raise NotImplementedError
NotImplementedError

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


The purpose of definiing the method `__ne__` as we have is to avoid writing the entire definition, we're using the same logic definied with `__eq__`.

Commutative addition: adding point A to B gives the same result as adding point B to point A. Elliptic curves use 'point addition' which adds two points that are on the curves to give a third point on also on the curve.

lines on a curve can intercept once or three times. It's also possible to intercept two times, where one of the two interceptions is tangent to the curve. The other case is when the line is vertical, and therefore must intercet at two points.

Point addition can be not easily predictable.

Point addition satisfies identity, commutativity, associativity, invertibility.

In [13]:
from ecc import Point
p1 = Point(-1, -1, 5, 7)
p2 = Point(-1, 1, 5, 7)
inf = Point(None, None, 5, 7)
print(p1 + inf)
print(inf + p2)
print(p1 + p2)

Point(-1,-1)_5_7
Point(-1,1)_5_7


NotImplementedError: 

# Exercise 3

Handle the case where the two points are additive inverses. That is, they have the same `x`, but a different `y`, causing a vertical line. This should return the point at infinity.

#### Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_add0`

In [ ]:
# Exercise 3

reload(ecc)
run(ecc.PointTest("test_add0"))

# Exercise 4

For the curve \\(y^{2}\\)=\\(x^{3}\\)+5x+7, what is (2,5) + (-1,-1)?

In [ ]:
# Exercise 4

from ecc import Point

a = 5
b = 7
x1, y1 = 2, 5
x2, y2 = -1, -1

# (x1,y1) + (x2,y2)

# Exercise 5

Write the `__add__` method where \\(x_{1}\\)≠\\(x_{2}\\)

#### Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_add1`

In [ ]:
# Exercise 5

reload(ecc)
run(ecc.PointTest("test_add1"))

# Exercise 6

For the curve \\(y^{2}\\)=\\(x^{3}\\)+5x+7, what is (-1,-1) + (-1,-1)?

In [ ]:
# Exercise 6

from ecc import Point

a = 5
b = 7
x1, y1 = -1, -1
# (-1,-1) + (-1,-1)

# Exercise 7

Write the `__add__` method when \\(P_{1}\\)=\\(P_{2}\\).

#### Make [this test](/edit/code-ch02/ecc.py) pass: `ecc.py:PointTest:test_add2`

In [ ]:
# Exercise 7

reload(ecc)
run(ecc.PointTest("test_add2"))